In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal as mvn
import seaborn as sns


MNIST_data_train = pd.read_csv('/content/drive/MyDrive/Data Folder/MNIST_train.csv', index_col ="index")
MNIST_data_test = pd.read_csv('/content/drive/MyDrive/Data Folder/MNIST_test.csv', index_col = "index")

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
MNIST_data_train
#data

,Unnamed: 0,labels,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,...,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,147,252,42,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,59995,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
59996,59996,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
59997,59997,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Sort the dataset into "X" and "y" categories, and normalize the "X" data to a scale of 0 to 1.

In [6]:
X = MNIST_data_train.to_numpy()
y_train = X[:, 1]
X_train = X[:, 2:]
X_train = X_train / 255


In [7]:
X = MNIST_data_test.to_numpy()
y_test = X[:, 1]
X_test = X[:, 2:]
X_test = X_test / 255

In [8]:
X_train[10]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

Naive Bayes algorithm

In [9]:
class GaussNB():
  def fit(self, X, y, epsilon=2e-3):
    self.likelihoods = dict()
    self.priors = dict()
    self.K = set(y.astype(int))
    
    for k in self.K:
      X_k = X[y == k, :]
      self.likelihoods[k] = {"mean": X_k.mean(axis=0), "cov": X_k.var(axis=0) + epsilon}
      self.priors[k] = len(X_k)/len(X)

  def predict(self, X):
    N, D = X.shape
    P_hat = np.zeros((N, len(self.K)))

    for k, l in self.likelihoods.items():
      P_hat[:, k] = mvn.logpdf(X, l['mean'], l['cov']) + np.log(self.priors[k])
    return P_hat.argmax(axis=1)
  

In [10]:
gnb = GaussNB()
gnb.fit(X_train,y_train)
y_hat_train = gnb.predict(X_train)



In [ ]:
def accuracy(y, y_hat):
  return np.mean(y == y_hat)

In [ ]:
accuracy(y_train, y_hat_train)

In [ ]:
y_hat_test = gnb.predict(X_test)

In [ ]:
accuracy(y_test, y_hat_test)

Gaussian Bayes algorithm

In [ ]:
from operator import matmul
class GaussBayes():
#We are no longer assuming independence of data!

  def fit(self, X, y, epsilon = 2e-3):

    self.likelihoods = dict()
    self.priors = dict()

    self.K = set(y.astype(int))

    for k in self.K:

      X_k = X[y == k, :]
      N_k,D = X_k.shape  #extracting dimensions from 'shape'
      mu_k = X_k.mean(axis = 0)

      self.likelihoods[k] = {"mean": X_k.mean(axis=0), "cov": (1/(N_k - 1)) * np.matmul((X_k-mu_k).T, X_k - mu_k) + epsilon * np.identity(D)}
      self.priors[k] = len(X_k)/len(X)

  def predict(self, X):
    N,D = X.shape
    P_hat = np.zeros((N, len(self.K)))

    for k, l in self.likelihoods.items():
      P_hat[:, k] = mvn.logpdf(X, l["mean"], l["cov"]) + np.log(self.priors[k])

    return P_hat.argmax(axis = 1)


  #P_hat = selecting from the probability of all class
  #Y_hat = selecting the max

In [ ]:
gbayes = GaussBayes()

In [ ]:
gbayes.fit(X_train, y_train)

In [ ]:
y_hat_gb = gbayes.predict(X_train)

In [ ]:
y_hat_gb_test = gbayes.predict(X_test)

In [ ]:
y_hat_gb_test

In [ ]:
accuracy(y_train, y_hat_gb)

In [ ]:
accuracy(y_test, y_hat_gb_test)

Plot confusion matrices, both to see the (quantitative) success rate and to observe where the program may be falling short.

In [ ]:
plt.figure(figsize = (12, 9))
y_actu = pd.Series(y_test, name = "Actual")
y_pred = pd.Series(y_hat_test, name = "Predicted")
cm = pd.crosstab(y_actu, y_pred)
ax = sns.heatmap(cm, annot = True, fmt = "d")
plt.ylabel("True Label")
plt.xlabel("Predicted Label")

In [ ]:
plt.figure(figsize = (12, 9))
y_actu = pd.Series(y_test, name = "Actual")
y_pred = pd.Series(y_hat_gb_test, name = "Predicted")
cm = pd.crosstab(y_actu, y_pred)
ax = sns.heatmap(cm, annot = True, fmt = "d")
plt.ylabel("True Label")
plt.xlabel("Predicted Label")

In [ ]:
F_matrix_y = np.arange(10)
F_matrix_y

Visualization of any X_train character

In [ ]:
X_train1 = X_train.reshape(60000, 28, 28)

In [ ]:
#X
#y

sample = 1001
image = X_train1[sample]
fig = plt.figure()
plt.imshow(image, cmap = 'Blues')
#plt.hist(X[y==1,0], alpha=0.5, bins=30)
#plt.hist(X[y==0,0], alpha=0.5, bins=30)
plt.show()

Analysis of y_train data

In [ ]:
plt.figure()
plt.hist(X[y_train==0,1], label="0", alpha=0.5, bins=30)
plt.hist(X[y_train==1,1], label="1", alpha=0.5, bins=30)
plt.hist(X[y_train==2,1], label="2", alpha=0.5, bins=30)
plt.hist(X[y_train==3,1], label="3", alpha=0.5, bins=30)
plt.hist(X[y_train==4,1], label="4", alpha=0.5, bins=30)
plt.hist(X[y_train==5,1], label="5", alpha=0.5, bins=30)
plt.hist(X[y_train==6,1], label="6", alpha=0.5, bins=30)
plt.hist(X[y_train==7,1], label="7", alpha=0.5, bins=30)
plt.hist(X[y_train==8,1], label="8", alpha=0.5, bins=30)
plt.hist(X[y_train==9,1], label="9", alpha=0.5, bins=30)
plt.show()

In [ ]:
number_count = []
for i in range(0,10):
  quant = list(y_train == i)
  number_count.append(quant.count(True))

number_count = np.array(number_count)
number_count

In [ ]:
number_count1 = []
for i in range(0,10):
  quant = list(y_test == i)
  number_count1.append(quant.count(True))

number_count1 = np.array(number_count1)
number_count1

Superimposing the data for each digit's character set to create a set of fundamental digit versions

In [ ]:
F_matrix = np.zeros((10, 784))

for i in range(0,10):
  for HW_id in range(0, 60000):
    if y_train[HW_id] == i:
      F_matrix[i, :] += X_train[HW_id, :]
  F_matrix[i, :] /= number_count[i]


In [ ]:
F_matrix.shape
F_matrix1 = F_matrix.reshape(10, 28, 28)

F_matrix1[6]

In [ ]:
image = F_matrix1[0]
fig = plt.figure()
plt.imshow(image, cmap = 'Blues')
plt.show()

In [ ]:
num = 10
images = F_matrix1[: num]
labels = F_matrix_y[: num]

In [ ]:
num_row = 2
num_col = 5
fig, axes = plt.subplots(num_row, num_col, figsize = (1.5*num_col, 2*num_row))
for i in range(num):
  ax = axes[i // num_col, i % num_col]
  ax.imshow(images[i], cmap = 'GnBu')
  ax.set_title('Label: {}'.format(labels[i]))
plt.tight_layout()
plt.show()